```markdown
Assignment 2
Nazmul Hasan Rabbi
CISC 0672: Data Visualization
```


## Project Setup
Importing required packages and loading data required for this assignment


In [2]:
# importing packages
import pandas as pd

### Exercise 1
Go to https://www.ncdc.noaa.gov/cag/city/time-series/ and retrieve time-series temperature data for four to six cities of your choice. To get data for a city, select *Average Temperature*, *All Months*, *Start Year*, *End Year*, *State*, and *City*. 

Use Excel to combine these datasets into a single file *cityTempsSeries.csv* with temperature data for your cities. This has the same format as the file read at the top of this notebook, but it has data for your chosen cities.

Then use the techniques used in this notebook above to create a dataframe with the format given above.


In [27]:
# waiting for dataset
data = pd.read_csv('./data/cityTempsSeries.csv')
data.head()

,Date,Cleveland,Miami,NYC,SF
0,195001,35.0,73.6,41.0,44.0
1,195002,27.7,70.5,30.8,49.0
2,195003,31.5,72.2,35.8,50.6
3,195004,41.6,71.4,48.1,54.3
4,195005,59.7,79.5,58.5,55.7


### Exercise 2
For this exercise, define the function 

    plot_one_month(frame, month, cities) 
    
that draws a line graph over all years for given *month* and list of *cities*. Assume *frame* has one of the two formats (`df` or `df2`) defined above, though for this exercise you might find it easier to assume the `df` format.

The following image results from the function call:

    plot_one_month(frame, 'Jan', ['Cleveland', 'Miami', 'NYC']).show()